# Carvana Tiramisu

## Imports

In [1]:
#from keras.optimizers import RMSprop
#from keras.losses import binary_crossentropy
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.regularizers import l2
import keras.backend as K
import numpy as np
import pandas as pd
from keras.layers.advanced_activations import PReLU
from sklearn.model_selection import train_test_split
from optimizers.AdamAccumulate import AdamAccumulate
from submit import generate_submit
import utils
from keras.layers import AveragePooling2D
import tensorflow as tf
from models.tiramisu import Tiramisu
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
%matplotlib inline

Using TensorFlow backend.


## Preparing Data

In [2]:
input_size = 1024
train_path = "inputs/train/{}.jpg" 
train_mask_path = "inputs/train_masks/{}_mask.gif"
df_train = pd.read_csv('inputs/train_masks.csv')
ids_train = df_train['img'].map(lambda s: s.split('.')[0])#[:3000]
ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.2, random_state=42)

print('Training on {} samples'.format(len(ids_train_split)))
print('Validating on {} samples'.format(len(ids_valid_split)))

bboxes = None
bbox_file_path = 'inputs/train_bbox.csv'
bboxes = utils.get_bboxes(bbox_file_path)

def train_generator(batch_size):
    return utils.train_generator(train_path, train_mask_path, ids_train_split, input_size, batch_size, bboxes)

def valid_generator(batch_size):
    return utils.valid_generator(train_path, train_mask_path, ids_valid_split, input_size, batch_size, bboxes)

Training on 4070 samples
Validating on 1018 samples


## Loss Functions

In [3]:
def dice_value(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

#def bce_dice_loss(y_true, y_pred):
#    return binary_crossentropy(y_true, y_pred) + (1 - dice_value(y_true, y_pred))

def weighted_bce_loss(y_true, y_pred, weights):
    return K.mean(tf.nn.weighted_cross_entropy_with_logits(y_true, y_pred, weights), axis=-1)

def weighted_dice_value(y_true, y_pred, weights):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    weights = K.flatten(weights)
    w2 = weights * weights
    return (2. * K.sum(w2 * intersection) + smooth) / (K.sum(w2 * y_true_f) + K.sum(w2 * y_pred_f) + smooth)

def weighted_bce_dice_loss(y_true, y_pred):
    a = AveragePooling2D(pool_size=(11, 11), strides=1, padding='same')(y_true)
    ind = K.cast(K.greater_equal(a, 0.01), 'float32') * K.cast(K.less_equal(a, 0.99), 'float32')
    ind = K.cast(ind, 'float32')
    weights = K.ones_like(a)
    w0 = K.sum(weights)
    weights = weights + ind*2
    w1 = K.sum(weights)
    weights = weights/w1*w0
    return  weighted_bce_loss(y_true, y_pred, weights) + (1 - weighted_dice_value(y_true, y_pred, weights))

## Create Model

In [8]:
model = Tiramisu((input_size, input_size, 3), filters=20, growth_rate=8, depth=7, #activation=lambda x: PReLU()(x),
                 regularizer=l2(1e-4), maxpool=True, upconv=True)
model.compile(optimizer=AdamAccumulate(accum_iters=32), loss=weighted_bce_dice_loss, metrics=[dice_value])
#model.compile(optimizer=RMSprop(1e-3, decay=1-0.99995), loss=bce_dice_loss, metrics=[dice_value])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 1024, 1024, 3) 0                                            
____________________________________________________________________________________________________
conv2d_86 (Conv2D)               (None, 1024, 1024, 20 560         input_3[0][0]                    
____________________________________________________________________________________________________
activation_166 (Activation)      (None, 1024, 1024, 20 0           conv2d_86[0][0]                  
____________________________________________________________________________________________________
batch_normalization_75 (BatchNor (None, 1024, 1024, 20 80          activation_166[0][0]             
___________________________________________________________________________________________

## Fit Model

In [ ]:
epochs = 100
batch_size = 1

run_name = utils.get_run_name('weights/{}.hdf5', 'tiramisu')
weights_path = 'weights/{}.hdf5'.format(run_name)

callbacks = [EarlyStopping(monitor='val_dice_value',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4,
                           mode='max'),
             ReduceLROnPlateau(monitor='val_dice_value',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4,
                               mode='max'),
             ModelCheckpoint(monitor='val_dice_value',
                             filepath=weights_path,
                             save_best_only=True,
                             save_weights_only=True,
                             mode='max'),
             TensorBoard(log_dir='logs/{}'.format(run_name), batch_size=batch_size)]

#model.load_weights('weights/best_weights.hdf5')
#K.set_value(model.optimizer.lr, 1e-4)

print('Starting run "{}"'.format(run_name))
model.fit_generator(generator=train_generator(batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(batch_size)),
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=valid_generator(batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(batch_size)))

Starting run "tiramisu-2017-09-06-0440"
Epoch 1/100
4070/4070 [==============================] - 3171s - loss: 0.5789 - dice_value: 0.9722 - val_loss: 0.7149 - val_dice_value: 0.8878
Epoch 2/100
4070/4070 [==============================] - 3149s - loss: 0.5149 - dice_value: 0.9873 - val_loss: 0.6995 - val_dice_value: 0.8949
Epoch 3/100
4070/4070 [==============================] - 3146s - loss: 0.5051 - dice_value: 0.9888 - val_loss: 0.7596 - val_dice_value: 0.8617
Epoch 4/100
1648/4070 [===========>..................] - ETA: 1761s - loss: 0.5017 - dice_value: 0.9893